In [3]:
from copy import deepcopy

# Project 1 - Sequence alignment

Genetic material such as DNA and proteins comes in sequences made up of different blocks. The study of these sequences and their correlation is invaluable to the field of biology. Since genetic material is likely to mutate over time, it is not easy to be certain of these correlations in most cases. In order to know if two or more sequences have a common origin, we must examine their similarity in a formal way, and be able to quantify it. Since the modifications of the sequences can introduce new blocks as well as delete some, the first step to comparing sequences is aligning them by making the similar regions match. Even then, high levels of similarity do not necessarily imply a common ancestor exists (homology). Our goal in this section is to create a tool that allows us to perform such alignments and give a numeric value of the chance of homology.


## ADTs

We will first look at an implementation of Abstract Data Types (ADTs) that will allow us to represent Sequences. 

### AminoAcid

The choice of genetic material is proteins. In the case of proteins, the blocks that make up the sequences are called amino acids. Therefore, they are the first thing we must be able to represent. The following tuple lists all amino acids and miscellaneous options.

In [4]:
#All the names of amino acids, plus gaps and some (partially) undetermined blocks
nameGroups = (
		("none/gap", "gap", "-"),
		("alanine", "ala", "A"),
		("cysteine", "cys", "C"),
		("aspartic acid", "asp", "D"),
		("glutamic acid", "glu", "E"),
		("phenylalanine", "phe", "F"),
		("glycine", "gly", "G"),
		("histidine", "his", "H"),
		("isoleucine", "ile", "I"),
		("lysine", "lys", "K"),
		("leucine", "leu", "L"),
		("methionine", "met", "M"),
		("asparagine", "asn", "N"),
		("proline", "pro", "P"),
		("glutamine", "gln", "Q"),
		("arginine", "arg", "R"),
		("serine", "ser", "S"),
		("threonine", "thr", "T"),
		("valine", "val", "V"),
		("tryptophan", "trp", "W"),
		("tyrosine", "tyr", "Y"),
		("asparagine/aspartic acid", "asx", "B"),
		("glutamine/glutamic acid", "glx", "Z"),
		("leucine/isoleucine", "xle", "J"),
		("selenocysteine", "sec", "U"),
		("pyrrolysine", "pyl", "O"),
		("undetermined", "und", "X")
	)

The following class allows us to create AminoAcid objects that represent any of the amino acids from the list. We can then compare, print, hash, test them with any of the defined methods. Do note that we can create them by copying other AminoAcids or by providing their name in any form (long for complete, medium for 3 chars or short for 1 char).

In [5]:
class AminoAcid:
	"""
	Represents one of the amino acids that can be found in genetic sequences.
	Can be one of the twenty-two amino acids, four undetermined combinations of possible amino acids, and gaps.
	"""
	#Tuple of tuples listing amino acid names
	_nameGroups = deepcopy(nameGroups)
	
	#Dictionary mapping name to id
	_nameDict = {nameGroups[id][i]:id for i in range(3) for id in range(len(nameGroups))}
	
	_nameModes = {"long":0, "medium":1, "short":2} #choices for name lenght
	_defaultNameMode = "short" #short name by default
	
	
	def __init__(self, aminoAcid):
		"""
		Creates an AminoAcid object representing one of the possible Amino Acids.
		aminoAcid can be the name of an amino acid, or an AminoAcid object (in which case a copy is created).
		"""
		self._id = None	#id of the amino acid within the name group
		
		if isinstance(aminoAcid, str):
			self._id = self.__getIdByName(aminoAcid) #id found from aminoAcid name
		elif isinstance(aminoAcid, AminoAcid):
			self._id = aminoAcid._id #copy of id
		else:			
			raise TypeError("aminoAcid must be a string or an AminoAcid object")
		
	
	@staticmethod
	def __getIdByName(name):
		try:
			return AminoAcid._nameDict[name] #get index of name mode
		except:
			raise ValueError("Could not find amino acid name {}".format(name))
			
	@staticmethod
	def getAllNames(nameMode=_defaultNameMode):
		try:
			nameIndex = AminoAcid._nameModes[nameMode] #get index of name mode
		except:
			raise TypeError("nameMode must be 'short', 'medium' or 'long'")
			
		for aa in AminoAcid._nameGroups[1:]: #we exclude the gap (first item)
			yield aa[nameIndex]
	
	#Representation
	def __repr__(self):
		nameIndex = AminoAcid._nameModes[self._defaultNameMode]
		return self._nameGroups[self._id][nameIndex] #default name mode
		
	def __str__(self):
		return self.getName() #default name mode
	
	def getName(self, nameMode=_defaultNameMode):
		try:
			nameIndex = AminoAcid._nameModes[nameMode] #get index of name mode
		except:
			raise TypeError("nameMode must be 'short', 'medium' or 'long'")
		
		return self._nameGroups[self._id][nameIndex]
	
	#Comparison		
	def __eq__(self, other):
		return self._id == other._id
	
	def __ne__(self, other):
		return self._id != other._id
		
	def __gt__(self, other):
		return self._id > other._id
		
	def __ge__(self, other):
		return self._id >= other._id
		
	def __lt__(self, other):
		return self._id < other._id
	
	def __le__(self, other):
		return self._id <= other._id
		
	def isGap(self):
		return self._id == 0
	
	
	#Hashing
	def __hash__(self):
		return hash(self._id)

Let's test it a little bit.

In [6]:
a1 = AminoAcid("A")
print(a1)
a2 = AminoAcid(a1)
print(a1 == a2)
a3 = AminoAcid("K")
print(a3.getName("long"))

A
True
lysine


### Sequence

That's good and all, but we don't want to align single amino acids do we ? What we want to align is sequences. We can view a Sequence as an ordered list of AminoAcid objects. It should be iterable : we'll want to go through, access, and count its items. We'll also want to change them, insert new ones, delete some, and do all that transparently, as we would with a list. Finally it would be useful to check whether a sequence contains some other sub-sequence or single amino acid. The following is a class that does just that.

In [7]:
class Sequence:
	"""
	Represents a sequence of amino acids.
	"""
	
	def __init__(self, aminoAcids=None, description=""):
		"""
		Creates a Sequence object that represents the amino acid sequence contained in aminoAcids.
		aminoAcids can be one of the following :
		- None, meaning the Sequence is empty (default)
		- an AminoAcid object
		- a string of X AminoAcid short (uppercase) names or 1 AminoAcid name
		- a list containing AminoAcid objects and/or strings of individual AminoAcid names
		"""
		
		self._nameMode = "short" #the way in which AA names are displayed
		self._separator = "" #how to separate AA names when displayed
		self._description = description #description of the sequence
		if self._description == "" and isinstance(aminoAcids, Sequence):
			self._description = aminoAcids.getDescription()
		#Format aminoAcids into a list of AminoAcid objects.
		self._aaList = self.__formatAAList(aminoAcids) #List of amino acids
	
	
	@staticmethod
	def __formatAAList(aminoAcids):
		"""Formats 'aminoAcids' into a list of AminoAcid objects."""
		
		#Sequence object's aaList is deep copied
		if isinstance(aminoAcids, Sequence):
			return deepcopy(aminoAcids._aaList)
		
		#None becomes an empty Sequence
		elif aminoAcids is None:
			return []
		
		#A string is converted to a list
		elif isinstance(aminoAcids, str):
			if aminoAcids.isupper(): #Multiple Amino Acids in short name mode
				return [AminoAcid(aa) for aa in aminoAcids]
			else: #A single Amino Acid with any name mode
				return [AminoAcid(aminoAcids)]
		
		#An AminoAcid is copied (from name) and put within a list
		elif isinstance(aminoAcids, AminoAcid):
			return [AminoAcid(aminoAcids)] #Copy constructor
		
		#A list is copied with all of its items converted to AminoAcid objects
		elif isinstance(aminoAcids, list):
			return [AminoAcid(aa) for aa in aminoAcids]
		
		#No more supported types
		else:
			raise TypeError("aminoAcids must be a Sequence, list, AminoAcid object, string or None")
	
	#Size and comparison
	def __len__(self):
		return len(self._aaList)
		
	def __gt__(self, other):
		return len(self) > len(other)
		
	def __lt__(self, other):
		return len(self) < len(other)
		
	def __ge__(self, other):
		return len(self) >= len(other)
		
	def __le__(self, other):
		return len(self) <= len(other)
	
	def __eq__(self, other):
		return len(self) == len(other) and all(a == b for a, b in zip(self, other))
	
	def __ne__(self, other):
		return not self == other
	
	#Iteration
	def __iter__(self):
		return iter(self._aaList)
	
	#Representation
	def __repr__(self):
		"""Representation"""
		return str(self)
		
	def __str__(self):
		"""String conversion"""
		return self._separator.join([aa.getName(self._nameMode) for aa in self])
		
	def getDescription(self):
		"""Returns the description of the sequence."""
		return self._description
	
	def setNameMode(self, newMode):
		"""Changes the name display mode to 'newMode'."""
		if newMode in ("long", "medium", "short"):
			self._nameMode = newMode
		else:
			raise ValueError("newMode must be 'long', 'medium' or 'short'")
	
	def setSeparator(self, newSep):
		"""Changes the string that separates each displayed AminoAcid."""
		self._separator = newSep
	
	#Item and slice manipulation	
	def __getitem__(self, key):
		"""Return a Sequence object containing copies of the items from the slice"""
		if isinstance(key, slice):
			return Sequence([self._aaList[index] for index in range(*key.indices(len(self)))])
		else:
			try:
				return self._aaList[key] #Return the item of index 'key'
			except:
				raise ValueError("key does not represent an index or slice")
		
	def __setitem__(self, key, value):		
		"""Set value for a slice of the sequence"""
		if isinstance(key, slice):
			for index in range(*key.indices(len(self))): #range(start, stop, step)
				self._aaList[index] = AminoAcid(value) #create copies
		else:
			try:
				self._aaList[key] = AminoAcid(value) #Set value for one item
			except:
				raise ValueError("key does not represent an index or slice")
		
	def __delitem__(self, key):
		"""Delete a slice of the sequence"""
		if isinstance(key, slice):
			start, stop, step = key.indices(len(self))
			del self._aaList[start:stop:step]
		else:
			try:
				del self._aaList[key] #Delete one item
			except:
				raise ValueError("key does not represent an index or slice")
	
	
	#Sequence modification
	def insert(self, subSequence, index=0):
		"""
		Inserts subSequence into the Sequence at index 'index' (default is 0).
		subSequence must be compatible with an AminoAcid list, as specified by __formatAAList.
		"""
		#We need a formatted sequence
		for aa in self.__formatAAList(subSequence):
			self._aaList.insert(index, aa)
			index += 1
			
	def extend(self, subSequence):
		"""
		Same as calling insert at the end of the Sequence.
		"""
		self.insert(subSequence, len(self))
		
			
	def remove(self, subSequence, startIndex=0):
		"""
		Removes the first occurrence of 'subSequence' in self, starting at index. Returns
		- start index of the sub-sequence within self (if it exists)
		- False if subSequence is not a sub-sequence of self
		"""
		lookupResult = self.hasSubSequence(subSequence, startIndex)
		if isinstance(lookupResult, int):
			del self[startIndex:startIndex+len(subSequence)]
		
		return lookupResult
	
			
	def delete(self, start=0, stop=None, step=None):
		"""
		Deletes Amino Acids between indexes start (included) and stop (excluded).
		If start is not specified, deletion will begin at index 0.
		If stop is not specified, deletion will stop after one item.
		"""
		if stop is None:
			stop = start + 1
		if step is None:
			step = 1
		del self[start:stop:step]
	
	
	#Lookup
	def __contains__(self, item):
		"""
		Returns True if item is contained in Sequence, False if not.
		"""
		return item in self._aaList
	
	def hasSubSequence(self, subSequence, startIndex=0):
		"""
		Checks if 'subSequence' is a sub-sequence of self, starting at startIndex. Returns
		- start index of the sub-sequence within self, if it exists
		- False if sequence is not a sub-sequence of self
		"""
		subLen = len(subSequence)
		endIndex = len(self)-subLen+1
		if endIndex < startIndex or startIndex < 0:
			raise ValueError("subSequence does not fit in sequence after startIndex")
			
		for i in range(startIndex, endIndex):
			if all(self[i+j] == subSequence[j] for j in range(subLen)):
				return i
		
		return False

Since we may not want to type every sequence we use by hand, we can also read them from files. The format chosen here is .fasta, but this can be adapted for any other format.

In [8]:
def loadFasta(path):
	"""
	Loads the fasta file located in 'path' and yields the Sequences it contains.
	"""
	with open(path, 'r') as fastaFile:
		newSequence = None
		for line in fastaFile:
			line_s = line.strip()
			if line_s != "" and line_s[0] == ">":
				if newSequence is not None:
					yield newSequence
				newSequence = Sequence(None, line_s[1:])
			else:
				newSequence.extend(line_s)
		if len(newSequence)>0:
			yield newSequence

Let's test a few of the capabilities of this class :

In [9]:
s = Sequence("ABX")
print(s)
s.extend("cysteine")
print(s)
print(AminoAcid("alanine") in s)
sCopy = Sequence(s) #This is a deep copy
sAlias = s #This is the same object
del s[1:3]
sAlias[0] = "K"
s.setSeparator("-")
s.setNameMode("long")
print(s, sAlias, sCopy, sep=", ")

sequences = [seq for seq in loadFasta("SH3-sequences.fasta")]
print(sequences[0])
print(sequences[0].hasSubSequence(Sequence("RLQIVNNTEGDWWLAHSLS"))) #Returns the index of the sub-sequence, if found

ABX
ABXC
True
lysine-cysteine, lysine-cysteine, ABXC
GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
26


### Score

Now that the easy part is done, we need to remember why we're here (it's sequence alignment, just in case). We can align two sequences any number of ways, but we're only interested in alignments that could represent two descendants of the same original sequence. Therefore, we need to assign a **score** value to each alignment, that will help us see how significative it is.

Because of the way mutations happen, all modifications should not be treated equally. Some amino acids are more related between them than others, therefore making them more likely to mutate into each other. Some changes are more or less likely to happen because of the very structure of the protein. Also, when an amino acid is inserted into or deleted from a sequence (thus creating a **gap** in the alignment), it usually is not the only one. This tells us that, not only should the score of the alignment depend on each pair that's aligned (meaning, on the same location), but it should also depend on how many gaps there are and how big they are.

The `Score` class allows us to give a numerical value to each possible pair of amino acids. These values can be set manually one by one, just as they can be loaded from files. The chosen format here was `.iij`.

In [10]:
class Score:
	"""
	Represents a scoring matrix, used to determine the score between two Amino Acids
	"""
	
	def __init__(self, path="", description="", ignore=None):
		"""
		Creates a Score object.
		If 'path' is provided, loads the Score values from an iij file.
		Otherwise, creates a Score for all possible AminoAcids with values 0.
		"""
		self._description = description
		self._ignore = Sequence(ignore)
		self._matrix = []
		self._aaOrder = {}
		self._aaSequence = Sequence()
		
		#If path is provided, load directly from iij file
		if path != "":
			with open(path, 'r') as file:
				foundAAOrder = False #Have we found the line with the amino acid values and order yet?
				for line in file:
					if line[0] != "#": #Comments
						
						if not foundAAOrder: #Read aa values and order
							for aa in line.split():
								self._aaSequence.extend(aa)
							self._aaOrder = {aa: index for aa, index in zip(self._aaSequence, range(len(self._aaSequence)))}
							foundAAOrder = True
						else: #Read matrix values
							self._matrix.append([int(v) for v in line.split()])
		
		#Otherwise initialize matrix with 0
		else:
			lineSize = 1
			for aa in AminoAcid.getAllNames():
				if AminoAcid(aa) not in self._ignore:
					self._aaSequence.extend(aa)
					self._aaOrder[self._aaSequence[-1]] = lineSize-1
					self._matrix.append([0 for i in range(lineSize)])
					lineSize += 1
				
	#Representation
	def __repr__(self):
		"""
		Representation.
		"""
		sepSize = 4
		result = ["---------- " + self._description + " ----------"]
		for values, aa in zip(self._matrix, self._aaSequence):
			tempstr = '{a!s:<{w}}'.format(a=aa, w=sepSize)
			for value in values:
				tempstr += '{v:<{w}}'.format(v=value, w=sepSize)
			result.append(tempstr)
		tempstr = " "*sepSize
		for aa in self._aaSequence :
			tempstr += '{a!s:<{w}}'.format(a=aa, w=sepSize)
		result.append("")
		result.append(tempstr)
		return "\n".join(result)
	
	#Scoring
	def setScore(self, aa1, aa2, score):
		"""
		Set the score assigned to AminoAcids 'aa1', 'aa2'.
		"""
		id1 = self._aaOrder[aa1]
		id2 = self._aaOrder[aa2]
		if id1 > id2:
			self._matrix[id1][id2] = score
		else:
			self._matrix[id2][id1] = score
		
	def getScore(self, aa1, aa2):
		"""
		Get the score assigned to AminoAcids 'aa1', 'aa2'.
		"""
		id1 = self._aaOrder[aa1]
		id2 = self._aaOrder[aa2]
		if id1 > id2:
			return self._matrix[id1][id2]
		else:
			return self._matrix[id2][id1]

Here's an example on how to load a `Score` object from a `.iij` file, and access the scores for certains pairs :

In [11]:
scoring = Score("blosum62.iij", "BLOSUM 62")
print(scoring)
a1, a2 = Sequence("HN") #That's call unpacking, pretty neat huh ?
print(a1, a2, " : ", scoring.getScore(a1, a2))

---------- BLOSUM 62 ----------
A   4   
R   -1  5   
N   -2  0   6   
D   -2  -2  1   6   
C   0   -3  -3  -3  9   
Q   -1  1   0   0   -3  5   
E   -1  0   0   2   -4  2   5   
G   0   -2  0   -1  -3  -2  -2  6   
H   -2  0   1   -1  -3  0   0   -2  8   
I   -1  -3  -3  -3  -1  -3  -3  -4  -3  4   
L   -1  -2  -3  -4  -1  -2  -3  -4  -3  2   4   
K   -1  2   0   -1  -3  1   1   -2  -1  -3  -2  5   
M   -1  -1  -2  -3  -1  0   -2  -3  -2  1   2   -1  5   
F   -2  -3  -3  -3  -2  -3  -3  -3  -1  0   0   -3  0   6   
P   -1  -2  -2  -1  -3  -1  -1  -2  -2  -3  -3  -1  -2  -4  7   
S   1   -1  1   0   -1  0   0   0   -1  -2  -2  0   -1  -2  -1  4   
T   0   -1  0   -1  -1  -1  -1  -2  -2  -1  -1  -1  -1  -2  -1  1   5   
W   -3  -3  -4  -4  -2  -2  -3  -2  -2  -3  -2  -3  -1  1   -4  -3  -2  11  
Y   -2  -2  -2  -3  -2  -1  -2  -3  2   -1  -1  -2  -1  3   -3  -2  -2  2   7   
V   0   -3  -3  -3  -1  -2  -2  -3  -3  3   1   -2  1   -1  -2  -2  0   -3  -1  4   
B   -2  -1  3   4   -3  0   

## Needleman-Wunsch Alignment

I know, I know : all these lines and still no alignment in sight. What does that title even mean ? Well, in 1970 Saul B. Needleman and Christian D. Wunsch came up with an effective algorithm for aligning sequences. It provides us with the alignments that get the best score, given certain conditions. It uses a scoring system like the one we've just covered, with the addition of gap penalties : negative values added to the score when the alignment creates (initial penalty) and extends (extended penalty) a gap. Here's an overview of its steps :
 * Create a matrix with enough rows to fit one sequence ($A$) and enough columns to fit the other ($B$) : each cell $(i, j)$ represents a possible alignment between two amino acids $A_i$ and $B_j$.
 * Add an initial row and column to the matrix, with values (scores) determined a certain number of ways. Keep in mind that these cells represent the beginning of an alignment where one sequence only has gaps, same with the last row and column for the end of the alignment.
 * Go through every cell in the matrix and calculate its score based on the previous (left, top, left and top) cells, using the following formula where $(V,W,S)_{i,j}$ are 3 values contained in cell $(i,j)$ of the matrix, $Score(A_i, B_j)$ is the score between amino acids $A_i$ and $B_j$, $I$ is the initial gap penalty and $E$ the extended gap penalty.
$$
V(i,j) = max
\left\{
    \begin{array}{ll}
      S(i-1, j) + I\\
      V(i-1, j) - E
    \end{array}
\right.
\quad
W(i,j) = max
\left\{
    \begin{array}{ll}
      S(i, j-1) + I\\
      V(i, j-1) - E
    \end{array}
\right.
\quad
S(i,j) = max
\left\{
    \begin{array}{ll}
      S(i-1, j-1) + Score(A_i, B_j)\\
      V(i, j)\\
      W(i, j)
    \end{array}
\right.
$$
 * Backtrack from some point of the matrix (the end of the alignment) to some other (the beginning), only passing by permitted cells. The cells allowed after cell $(i,j)$ are the ones where the value $S(i,j)$ comes from : 
     * left if $S(i,j)=W(i,j)$ : sequence $A$ has a gap
     * top if $S(i,j)=V(i,j)$ : sequence $B$ has a gap
     * diagonal if $S(i,j)=S(i-1,j-1) + Score(A_i, B_j)$ : sequences $A$ and $B$ are aligned
     
Different types of alignments can be obtained by tweaking this algorithm.
 * **Global alignments** aim to align both sequences completely. In order to do that, we initialize the first row and sequences with multiples of $I$ and $E$, thus giving us negative values matching the gap required to get there. Backtracking starts at the end of the matrix and ends at the beginning.
 * **Local alignments** aim to produce the alignment with the best score, whithout regard for their lenght. We do not initialize the first row and column : completing an alignment with only gaps has no interest score-wise. Backtracking starts at the highest value(s) in the matrix and ends as soon as we reach a value of $0$. Local **suboptimal alignments** can be found by clearing the values of the local optimal alignment in the matrix, reevaluating scores for further rows and columns, and backtracking again.
 * **Semiglobal alignments** are intended for a global-like alignment between sequences that only overlap partially, or that have great difference in size (one is included in the other). The first row and column are not initialized for the same reason as with local alignments. Backtracking starts and tha highest value(s) but ends when we reach either the first line or first column (therefore finishing one sequence).


The following is a class that allows us to represent two aligned sequences, along with information about the way they were aligned and the result. **Identity** is the number of equal amino acids that are aligned, **similarity** is the number of similar (meaning equal or with a non negative score) amino acids that are aligned.

In [12]:
class AlignedSequences:
	"""
	Represents two aligned sequences with some metadata about the alignemnt.
	"""
	def __init__(self, seqA, seqB, seqInter, score, identity, gaps, similarity, alignmentType):
		self.seqA = seqA #Aligned sequences
		self.seqB = seqB
		self.seqInter = seqInter
		self.score = score #Score of alignment
		self.identity = identity #Identity
		self.gaps = gaps #Gaps count
		self.similarity = similarity + identity
		self.alignmentType = alignmentType #Options used for alignemnt
		self.condensed = False
		self.chunkSize = 80 #Used for display
		
	def __repr__(self):
		"""
		Object representation.
		"""
		res = []
		res.append("---------- Alignment ----------")
		res.append("Type       : " + self.alignmentType)
		res.append("Score      : " + str(self.score))
		res.append("Identity   : " + str(self.identity) \
			+ " ({0:.2f}%)".format(100*(self.identity/len(self.seqA))))
		res.append("Similarity : " + str(self.similarity) \
			+ " ({0:.2f}%)".format(100*(self.similarity/len(self.seqA))))
		res.append("Gaps       : " + (str(self.gaps) if self.gaps > 0 else "None"))
		res.append("")
		res.append("Upper Sequence : " + self.seqA.getDescription())
		res.append("Lower Sequence : " + self.seqB.getDescription())
		res.append("")
		if self.condensed:
			return "\n".join(res)
		
		listA, listB, listI = [], [], []
		for a, i, b in zip(self.seqA, self.seqInter, self.seqB):
			listA.append(str(a))
			listB.append(str(b))
			listI.append(i)
			if len(listA) == self.chunkSize:
				res.extend(["".join(listA), "".join(listI), "".join(listB), "\n"])
				listA, listB, listI = [], [], []
		
		if len(listA) > 0:
			res.extend(["".join(listA), "".join(listI), "".join(listB), ""])

		return "\n".join(res)

In order to use this class, we must (finally) implement the actual alignment algorithm. This is done by the following class.

In [13]:
class AlignMatrix:
	"""
	Represents an alignment matrix, used to determine the alignment score between two sequences
	"""
	
	def __init__(self, scoreMatrix):
		"""
		Creates an AlignMatrix object that uses scoreMatrix as a scoring system between amino acids.
		"""
		self._scoreMatrix = scoreMatrix #scoring matrix to use
		
		self._colSeq = None #copies of sequences to align
		self._rowSeq = None
		
		self._alignMatrix = None #Used for alignment scores between sequences
		self._rowGapMatrix = None #used for affine gap penalty
		self._colGapMatrix = None
		self._originMatrix = None #Used for backtracking purposes
		
		self._alignMode = "" #Align mode (global, semiglobal, local)
		self._isSuboptimal = False #Is the alignment subobtimal ?
		
		self._maxAlignScore = 0 #Maximum score
		self._currentAlignScore = 0 #Score for this alignment
		
		self._maxScoreRows = [] #Index of maximum values from _alignMatrix
		self._maxScoreCols = []
		self._currentAlignPath = []
		self._bestAlignPath = [] #sequence of origins for alignment with best score
		
		self._iniGapPenalty = 0 #Initial gap penalty
		self._extGapPenalty = 0 #Extended gap penalty
		
		self._alignedSeqA = None #Aligned sequences (result of alignment)
		self._alignedSeqB = None
	
	
	def globalAlign(self, seqA, seqB, iniGapPenalty, extGapPenalty=None, semiGlobal=False):
		"""
		Returns all best global alignment between 'seqA' and 'seqB' with the provided
		initial and extended gap penalties.
		If 'semiGlobal' is True, allows alignment to discard the start and end of either sequence.
		(this allows for better alignments when sequences overlap only partially)
		"""
		if semiGlobal:
			self._alignMode = "semiglobal"
		else:
			self._alignMode = "global"
		
		#Initialize all data structures
		self.__initialize(seqA, seqB, iniGapPenalty, extGapPenalty)
		
		#Align in global mode, from the bottom right
		if self._alignMode == "global":
			self._currentAlignScore = self._alignMatrix[-1][-1]
			yield from self.__align(len(self._rowSeq), len(self._colSeq))
		else:
			self._currentAlignScore = self._maxAlignScore
			for row, col in zip(self._maxScoreRows, self._maxScoreCols):
				yield from self.__align(row, col)
	
	
	def localAlign(self, seqA, seqB, iniGapPenalty, extGapPenalty=None, subOptimal=False):
		"""
		Returns all best local alignment between 'seqA' and 'seqB' with the provided
		initial and extended gap penalties.
		If 'subOptimal' is True, looks for best suboptimal alignments as well.
		"""
		self._alignMode = "local"
		
		#Initialize all data structures
		self.__initialize(seqA, seqB, iniGapPenalty, extGapPenalty)
		
		#Align in local mode, from the maximum value (optimal)
		self._currentAlignScore = self._maxAlignScore
		for row, col in zip(self._maxScoreRows, self._maxScoreCols):
			yield from self.__align(row, col)
		
		#If suboptimal alignments are requested
		if subOptimal:
			self._isSuboptimal = True
			self.__clearBestPath() #Reevalute scores with best alignment reset to 0
			self._currentAlignScore = self._maxAlignScore
			#Align in local mode, from the new maximum value (sub-optimal)
			for row, col in zip(self._maxScoreRows, self._maxScoreCols):
				yield from self.__align(row, col)
	
	
	def __initialize(self, seqA, seqB, iniGapPenalty, extGapPenalty):
		"""
		Sets all initial values for required data structures.
		"""
		#Sequences
		if len(seqA)==0 or len(seqB)==0:
				raise ValueError("seqA and seqB cannot be empty")
		self._colSeq = Sequence(seqA)
		self._rowSeq = Sequence(seqB)
		
		#Alignments
		self._subOptimal = False
		self._alignSeqA = Sequence()
		self._alignSeqB = Sequence()
		self._alignInter = ""
		
		self._maxAlignScore = 0 #Maximum score found while aligning
		self._maxScoreRows = [] #Rows of maximum score
		self._maxScoreCols = [] #Columns of maximum score 
		
		self._currentAlignPath = [] #indexes of the current alignment
		self._bestAlignPath = [] #indexes of the alignment with the best score
		
		#Gap penalties
		self._iniGapPenalty = iniGapPenalty
		self._extGapPenalty = iniGapPenalty if extGapPenalty is None else extGapPenalty
		
		#Matrices
		self._alignMatrix = [[0 for i in range(len(self._colSeq)+1)] \
								for j in range(len(self._rowSeq)+1)]
		self._rowGapMatrix = deepcopy(self._alignMatrix)
		self._colGapMatrix = deepcopy(self._alignMatrix)
		
		self._originMatrix = [["" for i in range(len(self._colSeq)+1)] \
								for j in range(len(self._rowSeq)+1)]
		
		#Global alignment : first line and colunm have initial scores and origins
		if self._alignMode== "global":
			for i in range(1, max(len(self._colSeq), len(self._rowSeq))+1):	
				val = self._iniGapPenalty + self._extGapPenalty*(i-1)
				if i <= len(self._colSeq): #First row
					self._alignMatrix[0][i] = val
					self._rowGapMatrix[0][i] = val 
					self._colGapMatrix[0][i] = val
					self._originMatrix[0][i] = "L" #Origin is left
				
				if i <= len(self._rowSeq): #First column
					self._alignMatrix[i][0] = val
					self._rowGapMatrix[i][0] = val 
					self._colGapMatrix[i][0] = val
					self._originMatrix[i][0] = "T" #Origin is top					
			
		
		#Fill all matrices
		for row in range(1, len(self._rowSeq)+1):
			for col in range(1, len(self._colSeq)+1):
				self.__fill(row, col)
	
	
	def __fill(self, row, col):
		"""
		Calculates the score for value at row 'row' and column 'col'.
		"""
		#Row gap matrix
		initialGapScore = self._alignMatrix[row-1][col] + self._iniGapPenalty
		extendedGapScore = self._rowGapMatrix[row-1][col] + self._extGapPenalty
		self._rowGapMatrix[row][col] = max(initialGapScore, extendedGapScore)
		
		#Column gap matrix
		initialGapScore = self._alignMatrix[row][col-1] + self._iniGapPenalty
		extendedGapScore = self._colGapMatrix[row][col-1] + self._extGapPenalty
		self._colGapMatrix[row][col] = max(initialGapScore, extendedGapScore)
		
		#Align matrix
		deleteScore = self._rowGapMatrix[row][col]
		insertScore = self._colGapMatrix[row][col]
		matchScore = self._alignMatrix[row-1][col-1] + \
			self._scoreMatrix.getScore(self._rowSeq[row-1], self._colSeq[col-1])
			
		maxScore = 0
		if self._alignMode == "local": #Local alignment
			self._alignMatrix[row][col] = maxScore = max(insertScore, deleteScore, matchScore, 0)
		else:	#Global alignment
			self._alignMatrix[row][col] = maxScore = max(insertScore, deleteScore, matchScore)
			
		#Max alignment scores
		if maxScore > self._maxAlignScore: #New max alignment
			self._maxAlignScore = maxScore
			self._maxScoreRows = [row]
			self._maxScoreCols = [col]
		elif maxScore == self._maxAlignScore: #Same max alignment
			self._maxScoreRows.append(row)
			self._maxScoreCols.append(col)
			
		#Origin matrix
		if insertScore == maxScore:
			self._originMatrix[row][col] += "L"
		if matchScore == maxScore:
			self._originMatrix[row][col] += "D"
		if deleteScore == maxScore:
			self._originMatrix[row][col] += "T"
	
	
	def __clearBestPath(self):
		"""
		Resets all scores from the first best alignment and then reevaluates the affected scores.
		A new maximum score will be found from these, allowing for new alignment lookups.
		"""
		self._maxAlignScore = 0 #New maximum will be found from modified values
		self._maxScoreRows = []
		self._maxScoreCols = []
		
		for row, col in self._bestAlignPath[::-1]: #Reverse best align path
			self._alignMatrix[row][col] = 0 #Reset scores
			self._colGapMatrix[row][col] = 0
			self._rowGapMatrix[row][col] = 0
			
			#Reevaluate scores for further rows and columns
			for furtherRow in range(row+1, len(self._rowSeq)+1):
				if (furtherRow, col) not in self._bestAlignPath: #Only if not part of best path
					self._originMatrix[furtherRow][col] = ""
					self.__fill(furtherRow, col)
			
			for furtherCol in range(col+1, len(self._colSeq)+1):
				if (row, furtherCol) not in self._bestAlignPath:
					self._originMatrix[row][furtherCol] = ""
					self.__fill(row, furtherCol)
					
			
		
	
	def __align(self, i, j, identity=0, gaps=0, similarity=0):
		"""
		Yields all best alignments starting from row i and column j.
		"""
		#Local alignments are complete when we reach a null score
		#Global alignments are complete when we reach the beginning of the matrix
		#Semiglobal alignments are complete when we reach an edge of the matrix
		if (self._alignMode == "local" and self._alignMatrix[i][j]==0) \
		or (self._alignMode == "global" and i==0 and j==0) \
		or (self._alignMode == "semiglobal" and (i==0 or j==0)):
			
			#Create AlignedSequences object as result
			result = AlignedSequences(Sequence(self._alignSeqA, self._colSeq.getDescription()), \
			Sequence(self._alignSeqB, self._rowSeq.getDescription()),self._alignInter[::-1], \
			self._currentAlignScore, identity, gaps, similarity, \
			self._alignMode + "-suboptimal"*self._isSuboptimal)
			
			#Remember first best alignment for subobtimal lookup
			if self._bestAlignPath == []:
				self._bestAlignPath = deepcopy(self._currentAlignPath)
			
			yield result
			
		else:
			for origin in self._originMatrix[i][j]:
				self._currentAlignPath.append((i,j))
				
				if origin == "T": #top
					self._alignSeqA.insert("gap", 0)
					self._alignSeqB.insert(self._rowSeq[i-1], 0)
					self._alignInter += " "
					yield from self.__align(i-1, j, identity, gaps+1, similarity)
					
				elif origin == "D": #diagonal
					if self._colSeq[j-1] == self._rowSeq[i-1]:
						identity += 1
						self._alignInter += ":"
					elif self._scoreMatrix.getScore(self._colSeq[j-1], self._rowSeq[i-1])>=0:
						similarity += 1
						self._alignInter += "."
					else:
						self._alignInter += " "
					self._alignSeqA.insert(self._colSeq[j-1], 0)
					self._alignSeqB.insert(self._rowSeq[i-1], 0)
					yield from self.__align(i-1, j-1, identity, gaps, similarity)
				
				elif origin == "L": #left
					self._alignSeqA.insert(self._colSeq[j-1], 0)
					self._alignSeqB.insert("gap", 0)
					self._alignInter += " "
					yield from self.__align(i, j-1, identity, gaps+1, similarity)
				
				else:
					raise ValueError("Origin must be T (top), D (diagonal) or L (left)")
				
				self._currentAlignPath.pop()
				self._alignSeqA.delete(0)
				self._alignSeqB.delete(0)
				self._alignInter = self._alignInter[:-1]

		
	#Representation
	def __repr__(self):
		"""
		Representation of the alignemnt matrix.
		"""
		if self._colSeq is None:
			return "No alignment done yet"
		
		result = []
		sepSize = 5
		rep = {"":"", "L": "_   ", "D": " \\  ", "T": "   |", "LD": "_\\  ","DT": " \\ |", "LT": "_  |", "LDT": "_\\ |"}
		#Amino Acid column sequence
		result.append(" "*2*sepSize)
		for aa in self._colSeq :
			result[-1] += '{a!s:<{w}}'.format(a=aa, w=sepSize)
		
		#First line of values (no amino acid)
		result.append(" ")
		for origin in self._originMatrix[0]:
			result[-1] += '{o:>{w}}'.format(o=rep[origin], w=sepSize)
		
		result.append(" "*sepSize)
		for value in self._alignMatrix[0]:
			result[-1] += '{v:<{w}}'.format(v=value, w=sepSize)
		
		#Amino Acid line sequence and values
		for values, origins, aa in zip(self._alignMatrix[1:], self._originMatrix[1:], self._rowSeq):
			result.append(" ")
			for origin in origins:
				result[-1] += '{o:>{w}}'.format(o=rep[origin], w=sepSize)
			result.append("")
			result[-1] += '{a!s:<{w}}'.format(a=aa, w=sepSize)
			for value in values:
				result[-1] += '{v:<{w}}'.format(v=value, w=sepSize)
		
		return "\n".join(result)

Here is the result of a global alignment between the first two sequences from "SH3-sequences.fasta", calculated by LALIGN with the BLOSUM62 scoring matrix, initial and extended gap penalties of $-12$ and $-2$ :
```
n-w opt:  69  Z-score: 152.4  bits: 31.6 E(1): 6.7e-25
global/global (N-W) score: 69; 29.0% identity (62.9% similar) in 62 aa overlap (1-62:1-58)

GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
      .: ::... .. .::::.:. :...:.   . :   .:. :. :.::.::.  .
---MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH
```

Next is the result from the AlignMatrix class

In [14]:
s = Score("blosum62.iij", "BLOSUM 62")
a = AlignMatrix(s)
sequences = [seq for seq in loadFasta("SH3-sequences.fasta")]

for align in a.globalAlign(sequences[0], sequences[1], -12, -2, False):
	print(align)

---------- Alignment ----------
Type       : global
Score      : 73
Identity   : 18 (29.03%)
Similarity : 39 (62.90%)
Gaps       : 4

Upper Sequence : sp|P12931|84-145
Lower Sequence : sp|P62993|1-58

GGVTTFVALYDYESRTETDLSFKKGERLQIVNNTEGDWWLAHSLSTGQTGYIPSNYVAPSDS
      .: ::... .. .::::.:. :...:.   . :   .:. :. :.::.::.  .  
---MEAIAKYDFKATADDELSFKRGDILKVLNEECDQNWYKAELN-GKDGFIPKNYIEMKPH



Besides the second one being much more readable, their output is pretty similar. There may be slight differences in the BLOSUM matrix used, responsible for the discrepancy between the scores.

Here is the result of a local alignment between the first two sequences from "maguk-sequences.fasta", calculated by LALIGN with the BLOSUM62 scoring matrix, initial and extended gap penalties of $-12$ and $-2$ :
```
Waterman-Eggert score: 2677;  1048.2 bits; E(1) <  0
69.5% identity (86.3% similar) in 767 aa overlap (153-904:61-817)

SHSHISPIKPTEA-VLPSPPTVPVIPVLPVPAENTVILP-TIPQANPPPVLVNTDSLETP
:..  .:   :.: ..:.   :  .:  :::...:  :  . :.  :  . .: :  :  
SQAGATPTPRTKAKLIPTGRDVGPVPPKPVPGKSTPKLNGSGPSWWPECTCTNRDWYEQ-

TYVNGTDADYEYEEITLERGNSGLGFSIAGGTDNPHIGDDSSIFITKIITGGAAAQDGRL
  :::.:. ..::::.::::::::::::::: ::::. :: .::::::: :::::.::::
--VNGSDGMFKYEEIVLERGNSGLGFSIAGGIDNPHVPDDPGIFITKIIPGGAAAMDGRL

RVNDCILRVNEVDVRDVTHSKAVEALKEAGSIVRLYVKRRKPVSEKIMEIKLIKGPKGLG
 ::::.:::::::: .:.::.::::::::: .::: :.::.:  : :::..:.:::::::
GVNDCVLRVNEVDVSEVVHSRAVEALKEAGPVVRLVVRRRQPPPETIMEVNLLKGPKGLG

FSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGKLQIGDKLLAVNNVCLEEVTHEEAVTA
::::::.::::::::::::.:::::::::.:::.:::::.::::::. :..: :::::..
FSIAGGIGNQHIPGDNSIYITKIIEGGAAQKDGRLQIGDRLLAVNNTNLQDVRHEEAVAS
      
LKNTSDFVYLKVAKPTSMYMNDGYAPPDITNSSSQPVDNHVSPSSFLGQTPA--------
::::::.:::::::: :...:: ::::: ... .  .:::.: .: ::   :        
LKNTSDMVYLKVAKPGSLHLNDMYAPPDYASTFTALADNHISHNSSLGYLGAVESKVSYP
     
-----SPARYSPVSKAVLGDDEITREPRKVVLHRGSTGLGFNIVGGEDGEGIFISFILAG
      :.::::. . .:.....::::::..::.:::::::::::::::::::.::::::
APPQVPPTRYSPIPRHMLAEEDFTREPRKIILHKGSTGLGFNIVGGEDGEGIFVSFILAG
  
GPADLSGELRKGDRIISVNSVDLRAASHEQAAAALKNAGQAVTIVAQYRPEEYSRFEAKI
::::::::::.::::.:::.:.:: :.:::::::::.:::.::::::::::::::::.::
GPADLSGELRRGDRILSVNGVNLRNATHEQAAAALKRAGQSVTIVAQYRPEEYSRFESKI
    
HDLREQMMNSSISSGSGSLRTSQKRSLYVRALFDYDKTKDSGLPSQGLNFKFGDILHVIN
:::::::::::.::::::::::.:::::::::::::.:.:: ::::::.:..::::::::
HDLREQMMNSSMSSGSGSLRTSEKRSLYVRALFDYDRTRDSCLPSQGLSFSYGDILHVIN
   
ASDDEWWQARQVTPDGESDEVGVIPSKRRVEKKERARLKTVKFNSKTRDKGEIPDDMGSK
:::::::::: ::: :::...::::::.:::::::::::::::...:   : : ..    
ASDDEWWQARLVTPHGESEQIGVIPSKKRVEKKERARLKTVKFHART---GMIESNRDFP
   
GLKHVTSNASDSESSYRGQEEYVLSYEPVNQQEVNYTRPVIILGPMKDRINDDLISEFPD
::    :.   . .. .:::. .::::::..::..:.::::::::::::.::::::::: 
GL----SDDYYGAKNLKGQEDAILSYEPVTRQEIHYARPVIILGPMKDRVNDDLISEFPH
  
KFGSCVPHTTRPKRDYEVDGRDYHFVTSREQMEKDIQEHKFIEAGQYNNHLYGTSVQSVR
::::::::::::.:: ::::.:::::.::::::::::..:::::::.:..:::::.::::
KFGSCVPHTTRPRRDNEVDGQDYHFVVSREQMEKDIQDNKFIEAGQFNDNLYGTSIQSVR
    
EVAEKGKHCILDVSGNAIKRLQIAQLYPISIFIKPKSMENIMEMNKRLTEEQARKTFERA
 :::.::::::::::::::::: ::::::.:::::::.: .::::.: : :::.: ...:
AVAERGKHCILDVSGNAIKRLQQAQLYPIAIFIKPKSIEALMEMNRRQTYEQANKIYDKA
 
MKLEQEFTEHFTAIVQGDTLEDIYNQVKQIIEEQSGSYIWVPAKEKL
::::::: :.::::::::.::.:::..:::::.::: :::::. :::
MKLEQEFGEYFTAIVQGDSLEEIYNKIKQIIEDQSGHYIWVPSPEKL  
```

In [15]:
s = Score("blosum62.iij", "BLOSUM 62")
a = AlignMatrix(s)
sequences = [seq for seq in loadFasta("maguk-sequences.fasta")]

for align in a.localAlign(sequences[0], sequences[1], -12, -2, False):
	align.chunkSize = 60
	print(align)
	break

---------- Alignment ----------
Type       : local
Score      : 2689
Identity   : 534 (69.62%)
Similarity : 663 (86.44%)
Gaps       : 25

Upper Sequence : sp|Q12959|1-904
Lower Sequence : sp|Q92796|DLG3_HUMAN Disks large homolog 3 OS=Homo sapiens GN=DLG3 PE=1 SV=2

SHSHISPIKPTEA-VLPSPPTVPVIPVLPVPAENTVIL-PTIPQANPPPVLVNTDSLETP
:..  .:   :.: ..:.   :  .:  :::...:  :  . :.  :  . .: :  :  
SQAGATPTPRTKAKLIPTGRDVGPVPPKPVPGKSTPKLNGSGPSWWPECTCTNRDWYEQ-


TYVNGTDADYEYEEITLERGNSGLGFSIAGGTDNPHIGDDSSIFITKIITGGAAAQDGRL
  :::.:. ..::::.::::::::::::::: ::::. :: .::::::: :::::.::::
--VNGSDGMFKYEEIVLERGNSGLGFSIAGGIDNPHVPDDPGIFITKIIPGGAAAMDGRL


RVNDCILRVNEVDVRDVTHSKAVEALKEAGSIVRLYVKRRKPVSEKIMEIKLIKGPKGLG
 ::::.:::::::: .:.::.::::::::: .::: :.::.:  : :::..:.:::::::
GVNDCVLRVNEVDVSEVVHSRAVEALKEAGPVVRLVVRRRQPPPETIMEVNLLKGPKGLG


FSIAGGVGNQHIPGDNSIYVTKIIEGGAAHKDGKLQIGDKLLAVNNVCLEEVTHEEAVTA
::::::.::::::::::::.:::::::::.:::.:::::.::::::. :..: :::::..
FSIAGGIGNQHIPGDNSIYITKIIEGGAAQKDGRLQIGDRLLAVNNTNLQDVRHEEA

Once more, alignments are quite similar in terms of gap locations, scores and identity/similarity. The results suggest that these sequences might be related, given their high identity and similarity.

For each other pair of sequences (condensed results), we don't get such high changes of homology, as suggested by these condensed results :

In [16]:
s = Score("blosum62.iij", "BLOSUM 62")
a = AlignMatrix(s)
sequences = [seq for seq in loadFasta("maguk-sequences.fasta")]

for i in range(len(sequences)-1):
	for j in range(i+1, len(sequences)):
		for align in a.localAlign(sequences[i], sequences[j], -12, -2, False):
			align.condensed = True
			print(align)
			break

---------- Alignment ----------
Type       : local
Score      : 2689
Identity   : 534 (69.62%)
Similarity : 663 (86.44%)
Gaps       : 25

Upper Sequence : sp|Q12959|1-904
Lower Sequence : sp|Q92796|DLG3_HUMAN Disks large homolog 3 OS=Homo sapiens GN=DLG3 PE=1 SV=2

---------- Alignment ----------
Type       : local
Score      : 369
Identity   : 113 (26.84%)
Similarity : 263 (62.47%)
Gaps       : 26

Upper Sequence : sp|Q12959|1-904
Lower Sequence : sp|O14936|1-926

---------- Alignment ----------
Type       : local
Score      : 168
Identity   : 33 (36.26%)
Similarity : 65 (71.43%)
Gaps       : 5

Upper Sequence : sp|Q12959|1-904
Lower Sequence : sp|Q86UL8|1-1455

---------- Alignment ----------
Type       : local
Score      : 428
Identity   : 117 (28.19%)
Similarity : 259 (62.41%)
Gaps       : 21

Upper Sequence : sp|Q92796|DLG3_HUMAN Disks large homolog 3 OS=Homo sapiens GN=DLG3 PE=1 SV=2
Lower Sequence : sp|O14936|1-926

---------- Alignment ----------
Type       : local
Score      :